In [1]:
# face detection for the 5 Celebrity Faces Dataset
# !pip install Pillow
# !pip install matplotlib
import os
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
import time

abs_path = "/raid/Data/Master_Dataset_Elvin_Mrunal/Facebook_created_dataset_Face_extracted_Real+Fake/train/"
new_path = "/scratch/ysolanki/SiameseFacebook/MasterDataset/"

if not os.path.exists(new_path):
    os.makedirs(new_path)

def extract_face(filename):
    image = Image.open(filename)
    image = image.resize((160, 160))
    face_array = asarray(image)
    return face_array
    
# load images and extract faces for all images in a directory
def load_faces(directory):
    faces = list()
    # enumerate file
    count = 0
    start = time.time()
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        face = extract_face(path)
#         if not face:
#             continue
        # store
        faces.append(face)
        
        count += 1
        if count %1000 == 0:
            end = time.time()
            print(face.shape)
            print("No. of images done: ", count, " and time taken is ", (end-start)/60)
            start = time.time()
    return faces

# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in listdir(directory):
        # path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        print(subdir)
        # load all faces in the subdirectory
        faces = load_faces(path)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>loaded %d examples for class: %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)
 
# load train dataset
trainX, trainy = load_dataset(abs_path)
print(trainX.shape, trainy.shape)
# load test dataset
# testX, testy = load_dataset('5-celebrity-faces-dataset/val/')
# save arrays to one file in compressed format
savez_compressed(new_path + 'Master-dataset-from-elvin-images-160x160.npz', trainX, trainy)

ModuleNotFoundError: No module named 'PIL'

In [6]:
file = "Master-dataset-from-elvin-images-160x160.npz"

array([[[90, 66, 64],
        [90, 66, 64],
        [89, 65, 61],
        ...,
        [41, 30, 26],
        [41, 30, 26],
        [41, 30, 26]],

       [[90, 66, 64],
        [89, 65, 63],
        [89, 65, 61],
        ...,
        [41, 30, 26],
        [41, 30, 26],
        [41, 30, 26]],

       [[89, 65, 63],
        [89, 65, 63],
        [88, 64, 60],
        ...,
        [41, 30, 26],
        [41, 30, 26],
        [41, 30, 26]],

       ...,

       [[46, 30, 30],
        [44, 28, 29],
        [42, 26, 27],
        ...,
        [73, 58, 55],
        [76, 61, 58],
        [77, 62, 59]],

       [[46, 30, 30],
        [44, 28, 29],
        [41, 25, 26],
        ...,
        [73, 58, 55],
        [75, 60, 57],
        [77, 62, 59]],

       [[45, 29, 29],
        [43, 27, 28],
        [41, 25, 26],
        ...,
        [72, 57, 54],
        [75, 60, 57],
        [76, 61, 58]]], dtype=uint8)

In [8]:
if not os.path.exists(new_path):
    os.makedirs(new_path)
savez_compressed(new_path + 'Master-dataset-from-elvin-images.npz', trainX, trainy)

In [4]:
cd /raid/Data/Master_Dataset_Elvin_Mrunal/Facebook_created_dataset_Face_extracted_Real+Fake/train

/raid/Data/Master_Dataset_Elvin_Mrunal/Facebook_created_dataset_Face_extracted_Real+Fake/train


In [5]:
ls

fake/  real/


In [2]:
# calculate a face embedding for each face in the dataset using facenet
# !pip install keras
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
import time
import numpy as np

new_path = "/scratch/ysolanki/SiameseFacebook/MasterDataset/"


# # get the face embedding for one face
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    print(face_pixels.shape)
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = model.predict(samples)
    return yhat[0]


# load the face dataset
np.load.__defaults__=(None, True, True, 'ASCII')
data = load(new_path + 'Master-dataset-from-elvin-images-160x160.npz')
np.load.__defaults__=(None, False, True, 'ASCII')
print(data)
trainX, trainy = data['arr_0'], data['arr_1']
print('Loaded: ', trainX.shape, trainy.shape)
# load the facenet model
model = load_model('facenet_keras.h5')
print('Loaded Model')
# convert each face in the train set to an embedding

# embedder = FaceNet()

newTrainX = list()
count = 0
start = time.time()
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
#     pixels = expand_dims(face_pixels, axis=0)
#     print(pixels)
#     print(pixels.shape)
#     embedding = embedder.embeddings(pixels)
    newTrainX.append(embedding)
    count += 1
    if count%1000 == 0:
        end = time.time()
        print("No. of images done: ", count, " and time taken is ", (end-start)/60)
        start = time.time()
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
# convert each face in the test set to an embedding
# newTestX = list()
# for face_pixels in testX:
#     embedding = get_embedding(model, face_pixels)
#     newTestX.append(embedding)
# newTestX = asarray(newTestX)
# print(newTestX.shape)
# save arrays to one file in compressed format
savez_compressed(new_path + 'Master-dataset-elvin-faces-embeddings-160x160.npz', newTrainX, trainy)

ModuleNotFoundError: No module named 'keras'

In [8]:
import numpy as np

new_path = "/scratch/ysolanki/SiameseFacebook/MasterDataset/"
file1 = "Master-dataset-elvin-faces-embeddings-160x160.npz"
file2 = "Master-dataset-from-elvin-images-160x160.npz"

data = np.load(new_path + file1)
print(data['arr_0'].shape)

(496186, 128)


In [20]:
# # ValueError: Object arrays cannot be loaded when allow_pickle=False

# import numpy as np
# # save np.load
# np_load_old = np.load

# # modify the default parameters of np.load
# np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# # call load_data with allow_pickle implicitly set to true
# data = np.load(new_path + 'Master-dataset-from-elvin-images.npz')

# # restore np.load for future normal usage
np.load = np_load_old